In [1]:
#Import Various Libraries, including Tweepy, a Python library for the Twitter API.
import os
import requests
import pandas as pd
from dotenv import load_dotenv
#%matplotlib inline
import tweepy

load_dotenv()

True

In [91]:
#Pass API Keys to Twitter API and build Tweepy API handler object.
consumer_key = os.getenv('TWITTER_API_KEY')
consumer_secret = os.getenv('TWITTER_SECRET_KEY')
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

In [92]:
#Creating a DataFrame for tweet search results data structure.
#Index:User, Columns=['Category','Time','Sentiment','Text']
dict_tweet_structure = {"twitter_user":[],"category":[],"time":[],"sentiment":[],"text":""}
df_tweets_found = pd.DataFrame(dict_tweet_structure)
df_tweets_found

,twitter_user,category,time,sentiment,text


In [102]:
#Define Search Term Library
dict_search_terms = {'bitcoin':['bitcoin','btc','#bitcoin','#btc',],
                     'generic':['cryptocurrency','crypto','blockchain', 'struggle']}
#Define Twitter User List
list_twitterers = ['joerogan','elonmusk']

In [36]:
#Function for creating search query string for tweepy.Cursor(api.search).
def query_creator(list_keywords, twitter_user = "!", mood = "!"):
    query = "("
    for keyword in (list_keywords):
        query += keyword
        if keyword != list_keywords[-1]:
            query += " OR "
        else:
            query += ")"
    if twitter_user != "!":
        query += " from:" + twitter_user
    return query

In [32]:
#Function for creating search query string for tweepy.Cursor(api.search_full_archive).
def query_creator_full(list_keywords, twitter_user = "!", mood = "!"):
    query = " "
    query = query.join(list_keywords)
    #print(query)
    if twitter_user != "!":
        query += " from:" + twitter_user
        #print(query)
    return query

In [101]:
#This function is the core algo for tweet searching.
def tweet_search(string_query, date_from = "202001010000", date_to = "202101170000", number_tweets = 100):
    for tweets in tweepy.Cursor(api.search_full_archive, query=query, environment_name='CryptoSentimentFullArc', fromDate=date_since_pro, toDate=date_from_pro).items(100):
        screen_name = tweets.user.screen_name
        #for
        category = category_key(tweets.text ,dict_search_terms)
        yield [screen_name, category, str(tweets.created_at), "", tweets.text]

In [88]:
#Function for generating a string list of categories from the presence of keywords in text.
def category_key(text_block,dict_keywords):
    category_list = []
    output = " "
    contents = text_block.lower()
    for key in dict_keywords:
        for keyword in dict_keywords[key]:
            if keyword in contents:
                category_list.append(key)
    return output.join(category_list)

In [14]:

for tweet in tweepy.Cursor(api.search, q='cryptocurrency OR crypto OR blockchain OR struggle from:joerogan').items(1000):
    print(tweet.text + " " + str(tweet.created_at)+"\n")

Self imposed struggle is essential for a clear mind. @onnit https://t.co/mvd8yseLc6 2021-01-15 17:29:42+00:00



In [5]:
#Defunct Old Method
def tweet_gather(keywords, user_id):
    #contents = ""
    for tweet in tweepy.Cursor(api.user_timeline, id=user_id).items():
        """if tweet.truncated == True:
            contents = tweet.full_text
        else:
            contents = str(tweet.text).lower()"""
        
        contents = tweet.text.lower()
        #contents = contents.lower() 
        for word in keywords:           
            if word.lower() in contents:
                #print(tweet.text + " " + str(tweet.created_at)+"\n")
                """Here we can append to a pd.DataFrame that collects the twitter handle, keywords used, keyword category
                   for ex:'bitcoin' for hits on 'bitcoin' or 'btc', as well as any sentiment rating we create.
                   Ulitmately we can then export that DataFrame to csv file."""
                yield tweet.text, str(tweet.created_at)

In [6]:
"""Proposed structure for target terms = [{'bitcoin':['bitcoin','btc ','#bitcoin','#btc']}
                                         {'generic':['cryptocurrency','crypto','words about crypto']}]"""
words_crypto = ['Bitcoin',]#'Etherium','btc ','crypto','Litecoin','Eth ','python']
user_handle = 'elonmusk'
tweet_tuple = tweet_gather(words_crypto,user_handle)
for stuff in tweet_tuple:
    print(stuff)

('Bitcoin is almost as bs as fiat money', '2020-12-20 09:24:37+00:00')
('Bitcoin is my safe word', '2020-12-20 08:21:25+00:00')
('@Maisie_Williams 🎶 Toss a bitcoin to ur Witcher 🎶', '2020-11-16 22:02:51+00:00')
('@jk_rowling I still only own 0.25 Bitcoins btw', '2020-05-15 22:51:44+00:00')
('@jk_rowling Pretty much, although massive currency issuance by govt central banks is making Bitcoin Internet 👻 mone… https://t.co/pczxGWBebT', '2020-05-15 22:03:01+00:00')
('@Bitcoin How much for some anime Bitcoin? https://t.co/itqRslFNcb', '2020-05-01 22:44:56+00:00')
('@Bitcoin 🤣🤣', '2020-05-01 22:34:24+00:00')


In [97]:
#Testing Query Creator
dict_search_terms = {'bitcoin':['bitcoin','btc','#bitcoin','#btc',],
                     'generic':['cryptocurrency','crypto','blockchain', 'struggle']}
#print(query_creator(dict_search_terms[0]['bitcoin']))
#print(dict_search_terms['generic'])
query = query_creator(dict_search_terms['generic']+dict_search_terms['bitcoin'], 'joerogan')
#query = " "
#query = query.join(dict_search_terms['generic'])
#query += " from:joerogan"
#query += " -is:retweet"
date_since_pro = "202001140000"
date_from_pro = "202101170000"
print(query)

(cryptocurrency OR crypto OR blockchain OR struggle OR bitcoin OR btc OR #bitcoin OR #btc) from:joerogan


In [98]:
#Testing primary search and DataFrame append.
#for tweet in tweepy.Cursor(api.search_full_archive, query=query, environment_name='CryptoSentimentFullArc', fromDate=date_since_pro, toDate=date_from_pro).items(100):
   # print(tweet.text + " " + str(tweet.created_at)+"\n")
search_results = tweet_search(query)
for tweets in search_results:
    #print (tweet)
    series_result = pd.Series(tweets, index=df_tweets_found.columns)
    df_tweets_found = df_tweets_found.append(series_result, ignore_index=True)

In [99]:
df_tweets_found

,twitter_user,category,time,sentiment,text
0,elonmusk,bitcoin,2021-01-15 06:44:22+00:00,,Bitcoin is almost as bs as fiat money
1,elonmusk,bitcoin,2021-01-15 06:44:22+00:00,,Bitcoin is my safe word
2,elonmusk,,2021-01-15 06:44:22+00:00,,@trylolli @Maisie_Williams 👻 💵 \nhttps://t.co/...
3,elonmusk,bitcoin,2021-01-15 06:44:22+00:00,,@Maisie_Williams 🎶 Toss a bitcoin to ur Witcher 🎶
4,elonmusk,bitcoin,2021-01-15 06:44:22+00:00,,"@jk_rowling Pretty much, although massive curr..."
5,elonmusk,bitcoin,2021-01-15 06:44:22+00:00,,@Bitcoin How much for some anime Bitcoin? http...
6,elonmusk,bitcoin,2021-01-15 06:44:22+00:00,,@Bitcoin 🤣🤣
7,elonmusk,generic,2021-01-15 06:44:22+00:00,,@nichegamer The crypto scam level on Twitter i...
8,joerogan,generic,2021-01-15 06:44:22+00:00,,Self imposed struggle is essential for a clear...
9,joerogan,generic,2021-01-15 06:44:22+00:00,,My favorite tools of self-imposed struggle; th...


In [100]:
df_tweets_found.loc[4]['text']

'@jk_rowling Pretty much, although massive currency issuance by govt central banks is making Bitcoin Internet 👻 mone… https://t.co/pczxGWBebT'

In [7]:
for tweet in tweepy.Cursor(api.user_timeline, id='elonmusk').items():
    if 'bitcoin' in tweet.text:
        print(tweet.text + " " + str(tweet.created_at)+"\n")

@Maisie_Williams 🎶 Toss a bitcoin to ur Witcher 🎶


In [55]:
for tweet in tweepy.Cursor(api.user_timeline, id="elonmusk").items(100):
    if 'Python' in tweet.text:
        print(tweet.text)
        print(tweet.user['id_str'])

Monty Python is amazing
https://t.co/UJq94IWT88


TypeError: 'User' object is not subscriptable